In [31]:
%cd /content
!rm -r gpt2-ml

In [32]:
%cd /content/
!git clone https://github.com/imcaspar/gpt2-ml
%cd /content/gpt2-ml

In [33]:
!python3 -m pip install regex jsonlines 
!python3 -m pip install -U tqdm

In [34]:
import os
from google.colab import auth
from googleapiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from tqdm import tqdm


import os  
import io

auth.authenticate_user()
drive_service = build('drive', 'v3')

STEP_NUMBER = 100000
model_type = 'mega'
model_path = '/content/gpt2-ml/models'
model_dir = os.path.join(model_path, model_type)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

local_file_ids = ['1n_5-tgPpQ1gqbyLPbP1PwiFi2eo7SWw_']
local_filenames = ['.data-00000-of-00001']
for ext, id_ in zip(local_filenames, local_file_ids):
    ext = str(STEP_NUMBER) + ext
    filename = '%s/%s/model.ckpt-%s' % (model_path, model_type, ext)
    request = drive_service.files().get_media(fileId=id_)
    with open(filename, 'wb') as f:
      downloader = MediaIoBaseDownload(f, request, chunksize=100*1024*1024)
      done = False
      pbar = tqdm(total=100, desc='%s' % ext)
      progress = 0
      while done is False:
        status, done = downloader.next_chunk()
        new_progress = int(status.progress() * 100)
        pbar.update(new_progress - progress)
        progress = new_progress
      pbar.close()

!wget -q --show-progress https://github.com/imcaspar/gpt2-ml/releases/download/v0.5/model.ckpt-100000.index -O models/mega
!wget -q --show-progress https://github.com/imcaspar/gpt2-ml/releases/download/v0.5/model.ckpt-100000.meta -O models/mega

In [42]:
!git fetch && git checkout dev && git pull

In [0]:
#THis option lets you type from colab, but you can't enter new lines
!PYTHONPATH=$(pwd) python3 scripts/contextual_generate_cli.py -model_config_fn configs/mega.json -model_ckpt models/mega/model.ckpt-100000 -samples 10